# DegreeDiscountIC (a.k.a the ORACLE)

In [1]:
import networkx as nx
import numpy as np
from numpy.random import default_rng
rng = default_rng(12345)
import matplotlib.pyplot as plt
import sys; sys.path.append("model-independent-cascade\src")
from influence import influence_count
from scipy.stats import norm

Generating a random graph to test-run the model

In [2]:
G = nx.fast_gnp_random_graph(600, 0.1, directed=True)
edges = list(G.edges())

In [3]:
def sample_edges(edges, C, P_hat, O):
    epo = [(edges[i], P_hat[i], O[i]) for i in range(len(edges))]
    E = [[] for i in range(10)]
    for i in range(len(edges)):
        index = int(epo[i][1]/0.1)
        E[index].append(epo[i])
    E_hat = []
    for i in range(10):
        E[i].sort(key=lambda tuple: tuple[2], reverse=True)
        c_i = int(len(E[i])/(len(edges))*C)+1
        E_hat = E_hat + E[i][:c_i]

    return E_hat

In [50]:
def oracle(G, k, P):
    S = set({})
    temp = list(nx.degree(G))
    dd = d = [d for (_,d) in temp]
    t = [0 for _ in range(len(d))]

    for _ in range(k):
        u = np.argmax(dd)
        dd[u] = -1
        S.add(u)
        neighbors = set(G.neighbors(u))
        for v in (neighbors-S):
            p = P[edges.index((u, v))]
            t[v] += 1
            dd[v] = d[v] - 2*t[v] - (d[v] - t[v])*t[v]*p

    return list(S)

## The Algorithm

Coding it step-by-step:

Assigning the _ground-truth_ edge features and initializing the embeddings

In [5]:
for e in edges:
    n1, n2 = e
    alpha_t = rng.uniform(size=4)
    alpha_t /= np.linalg.norm(alpha_t)
    beta_t = rng.uniform(size=4)
    beta_t /= np.linalg.norm(beta_t)
    
    G.edges[n1,n2]['xt'] = np.concatenate((alpha_t, beta_t))

    alpha = rng.uniform(size=4)
    alpha /= np.linalg.norm(alpha)
    beta = rng.uniform(size=4)
    beta /= np.linalg.norm(beta)
    
    G.edges[n1,n2]['x'] = np.concatenate((alpha, beta))


In [18]:
sigma = 1
C = 80
z = 0.95
period = 20 
P_hat = [0 for _ in range(len(edges))]  
O = [0 for _ in range(len(edges))]
z = 2

In [19]:
Cedges = [a for (a,b,c) in sample_edges(edges, C, P_hat, O)]

In [20]:
len(Cedges)

81

In [21]:
f = lambda x1,x2: np.exp(-1)*np.linalg.norm(x1-x2)

In [28]:
X = np.array([G.edges[e]['x'] for e in Cedges])
Y = np.array([P_hat[edges.index(e)] for e in Cedges])

In [29]:
n = X.shape[0]
A = np.array([[f(x1,x2) for x2 in X] for x1 in X]).reshape((n, n)) + np.identity(n)

In [31]:
horizon = 200
period = 10

In [32]:
U = []
A_inv = np.linalg.inv(A)

for i in range(len(edges)):
    K_i = np.array([f(G.edges[edges[i]]['x'], x1) for x1 in X]).reshape(-1,1)
    m = np.dot(K_i.T, np.dot(A_inv, Y))
    var = 1 - np.dot(K_i.T, np.dot(A_inv, K_i))
    u = norm.cdf(m + z*var)
    U.append(u)

In [45]:
U = np.array(U).reshape(-1,1)

In [52]:
S = oracle(G, 30, U)

In [54]:
S[1]

518